#RoBERTa Small

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import RobertaTokenizer, TFRobertaModel

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Truth_Seeker/revised_cleaned_master_data.csv")

In [ ]:
df1, df2 = train_test_split(df, test_size=0.5, stratify=df['label'], random_state=42)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38230 entries, 67832 to 75180
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            38230 non-null  object
 1   text             38230 non-null  object
 2   label            38230 non-null  int64 
 3   subject          38230 non-null  object
 4   text_length      38230 non-null  int64 
 5   lemmatized_text  38229 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.0+ MB


In [ ]:
df1

,title,text,label,subject,text_length,lemmatized_text
67832,US envoy to Turkey says duration of visa servi...,ISTANBUL Reuters The US ambassador to Turkey ...,1,politicsNews,624,istanbul reuters u ambassador turkey said mond...
39802,Largest Pipeline in US Explodes Kills and Inju...,In September the Colonial pipeline the larges...,0,Unknown,184,september colonial pipeline largest pipeline u...
70346,Video Doctors Who Discovered Cancer Enzymes In...,Share on Facebook Nagalese prevents vitamin D ...,0,Unknown,2658,share facebook nagalese prevents vitamin produ...
45445,Nikki Haskell Learns the Social Cost of Suppor...,Nikki Haskell has packed many lives into her 7...,1,Unknown,12952,nikki haskell packed many life year real estat...
23954,Dilbert creator Scott Adams endorses Trump say...,Dilbert creator Scott Adams endorses Trump say...,0,Unknown,3943,dilbert creator scott adam endorses trump say ...
...,...,...,...,...,...,...
17371,Budget 2016 Obama moves left testing Republica...,President Obamas 2016 budget seeks higher spen...,1,Unknown,6500,president obamas budget seek higher spending e...
62087,Toys R Us Stores Offering Quiet Shopping Hours...,posted by Eddie Toys R Us stores in the UK are...,0,Unknown,2098,posted eddie toy r u store uk offering quiet h...
23376,Democratic Senator Franken to resign CNN citin...,WASHINGTON Reuters US Democratic Senator Al F...,1,politicsNews,294,washington reuters u democratic senator al fra...
34409,How to Plan a Safe Trip for Gay and Transgende...,Andy Hicks owner of HIX Inc a small health car...,1,Unknown,7987,andy hick owner hix inc small health care tech...


In [ ]:
# Encode labels
le = LabelEncoder()
df1['label'] = le.fit_transform(df1['label'])  # Assuming 'label' column contains 'fake' and 'real'

In [ ]:
# Splitting into train and test set
train_texts, test_texts, train_labels, test_labels = train_test_split(df1['lemmatized_text'].astype(str), df1['label'], test_size=0.2, random_state=42)

In [ ]:
# Initialize the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the texts
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),train_labels)).shuffle(1000).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),test_labels)).batch(32)

In [ ]:
# Load the pre-trained RoBERTa model
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

# Define the model
input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

roberta_output = roberta_model(input_ids, attention_mask=attention_mask)
output = tf.keras.layers.Dense(1, activation='sigmoid')(roberta_output[0][:, 0, :])  # Use the first token's output

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),loss='binary_crossentropy',metrics=['accuracy'])

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.embeddings.position_ids', 'lm_head.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [ ]:
# Train the model
history = model.fit(train_dataset, epochs=3, validation_data=test_dataset)

Epoch 1/3


956/956 [==============================] - 902s 904ms/step - loss: 0.2949 - accuracy: 0.8721 - val_loss: 0.1904 - val_accuracy: 0.9283
Epoch 2/3
956/956 [==============================] - 847s 886ms/step - loss: 0.1911 - accuracy: 0.9280 - val_loss: 0.1999 - val_accuracy: 0.9194
Epoch 3/3
956/956 [==============================] - 863s 903ms/step - loss: 0.1503 - accuracy: 0.9465 - val_loss: 0.2039 - val_accuracy: 0.9240


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

239/239 [==============================] - 67s 281ms/step - loss: 0.2039 - accuracy: 0.9240
Test Accuracy: 92.40%


#Distil RoBERTa

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch

In [ ]:
df['text_length'].describe()

,text_length
count,76461.000000
mean,3450.124456
std,3843.619582
min,2.000000
25%,1403.000000
50%,2464.000000
75%,4336.000000
max,130309.000000


In [ ]:
df1 = df[df['text_length'] <= 4336]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57347 entries, 0 to 76460
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            57347 non-null  object
 1   text             57347 non-null  object
 2   label            57347 non-null  int64 
 3   subject          57347 non-null  object
 4   text_length      57347 non-null  int64 
 5   lemmatized_text  57345 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.1+ MB


In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df1['lemmatized_text'].astype(str).tolist(),
    df1['label'].tolist(),
    test_size=0.1,
    random_state=42
)

In [ ]:
#Tokenized BERT
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#Tokenizing the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

train_labels = [int(label) for label in train_labels]
val_labels = [int(label) for label in val_labels]

train_data = Dataset.from_dict({'text': train_texts, 'label': train_labels})
val_data = Dataset.from_dict({'text': val_texts, 'label': val_labels})

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_val = val_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/51612 [00:00<?, ? examples/s]

Map:   0%|          | 0/5735 [00:00<?, ? examples/s]

In [ ]:
#Creating dataset
datasets = DatasetDict({'train': tokenized_train,'validation': tokenized_val})

In [ ]:
#Specifying training args
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=True,
    gradient_accumulation_steps=4
)

In [ ]:
#Creating model
trainer = Trainer(model=model,args=training_args,train_dataset=datasets['train'],eval_dataset=datasets['validation'])

In [ ]:
#Model training
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.707100,0.185721
2,0.409600,0.218699


TrainOutput(global_step=9675, training_loss=0.5978369984392664, metrics={'train_runtime': 2488.6157, 'train_samples_per_second': 62.218, 'train_steps_per_second': 3.888, 'total_flos': 2.050489357275955e+16, 'train_loss': 0.5978369984392664, 'epoch': 2.999147485080989})

In [ ]:
#Results
results = trainer.evaluate()
print(f"Results: {results}")

Results: {'eval_loss': 0.18211182951927185, 'eval_runtime': 26.9509, 'eval_samples_per_second': 212.794, 'eval_steps_per_second': 53.208, 'epoch': 2.999147485080989}


In [ ]:
import torch

predictions, labels, _ = trainer.predict(datasets['validation'])
predicted_labels = torch.argmax(torch.tensor(predictions), dim=1)

#Calculating accuracy
accuracy = (predicted_labels == labels).float().mean()
print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.9541412591934204


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

#Calculating precision, recall, and F1 score
precision = precision_score(labels, predicted_labels, average='weighted')
recall = recall_score(labels, predicted_labels, average='weighted')
f1 = f1_score(labels, predicted_labels, average='weighted')

# Printing results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.9545
Recall: 0.9541
F1 Score: 0.9541


In [ ]:
#Saving the model
trainer.save_model("./distilroberta-model")
tokenizer.save_pretrained("./distilroberta-model")

('./distilroberta-model/tokenizer_config.json',
 './distilroberta-model/special_tokens_map.json',
 './distilroberta-model/vocab.txt',
 './distilroberta-model/added_tokens.json')

#RoBERTa Large

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Truth_Seeker/revised_cleaned_master_data.csv")

In [ ]:
from sklearn.model_selection import train_test_split
df1, df2 = train_test_split(df, test_size=0.9, stratify=df['label'], random_state=42)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7646 entries, 2532 to 3594
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            7646 non-null   object
 1   text             7646 non-null   object
 2   label            7646 non-null   int64 
 3   subject          7646 non-null   object
 4   text_length      7646 non-null   int64 
 5   lemmatized_text  7646 non-null   object
dtypes: int64(2), object(4)
memory usage: 418.1+ KB


In [ ]:
# Tokenizing BERT
from transformers import AutoTokenizer
model_name = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1['lemmatized_text'].astype(str), df1['label'], test_size=0.2, random_state=42)

In [ ]:
max_length = 128
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=max_length, return_tensors='pt')
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=max_length, return_tensors='pt')

In [ ]:
import torch
from torch.utils.data import Dataset

class CustomNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        encoding = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
        encoding['labels'] = torch.tensor(self.labels[index])
        return encoding

#Creating train and test datasets
train_data = CustomNewsDataset(encodings=train_encodings, labels=y_train)
test_data = CustomNewsDataset(encodings=test_encodings, labels=y_test)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
# Training Arguments
training_args = TrainingArguments(output_dir='./results',evaluation_strategy="epoch",learning_rate=2e-5,per_device_train_batch_size=8,per_device_eval_batch_size=8,num_train_epochs=2,weight_decay=0.01,logging_dir='./logs',logging_steps=10,fp16=True,)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#Training Roberta
trainer = Trainer(model=model,args=training_args,train_dataset=train_dataset,eval_dataset=test_dataset)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


<ipython-input-6-a2ba469fc6d2>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.571500,0.426817
2,0.378100,0.401119


<ipython-input-6-a2ba469fc6d2>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-6-a2ba469fc6d2>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=1530, training_loss=0.5034879188911587, metrics={'train_runtime': 632.8799, 'train_samples_per_second': 19.328, 'train_steps_per_second': 2.418, 'total_flos': 2849846109057024.0, 'train_loss': 0.5034879188911587, 'epoch': 2.0})

In [ ]:
predictions = trainer.predict(test_dataset)
y_pred = torch.argmax(torch.tensor(predictions.predictions), axis=1)

<ipython-input-6-a2ba469fc6d2>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
print("Roberta Large Accuracy:", accuracy_score(y_test, y_pred)*100)
print(classification_report(y_test, y_pred))

Roberta Large Accuracy: 84.57516339869281
              precision    recall  f1-score   support

           0       0.76      0.92      0.83       633
           1       0.94      0.79      0.86       897

    accuracy                           0.85      1530
   macro avg       0.85      0.86      0.84      1530
weighted avg       0.86      0.85      0.85      1530

